<a href="https://colab.research.google.com/github/ManishaThakur007/RAGforPubmedQA/blob/main/Q1_fixed_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset #load pubmedqa data

In [ ]:
import pandas as pd
import numpy as np

pqa_artificial = pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_artificial/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
pqa_labeled =  pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_labeled/train-00000-of-00001.parquet")

In [ ]:
pqa_unlabeled = pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_unlabeled/train-00000-of-00001.parquet")

In [ ]:
pqa_artificial.head()

,pubid,question,context,long_answer,final_decision
0,25429730,Are group 2 innate lymphoid cells ( ILC2s ) in...,{'contexts': ['Chronic rhinosinusitis (CRS) is...,"As ILC2s are elevated in patients with CRSwNP,...",yes
1,25433161,Does vagus nerve contribute to the development...,{'contexts': ['Phosphatidylethanolamine N-meth...,Neuronal signals via the hepatic vagus nerve c...,yes
2,25445714,Does psammaplin A induce Sirtuin 1-dependent a...,{'contexts': ['Psammaplin A (PsA) is a natural...,PsA significantly inhibited MCF-7/adr cells pr...,yes
3,25431941,Is methylation of the FGFR2 gene associated wi...,{'contexts': ['This study examined links betwe...,We identified a novel biologically plausible c...,yes
4,25432519,Do tumor-infiltrating immune cell profiles and...,{'contexts': ['Tumor microenvironment immunity...,Breast cancer immune cell subpopulation profil...,yes


In [ ]:
pqa_labeled.head()

,pubid,question,context,long_answer,final_decision
0,21645374,Do mitochondria play a role in remodelling lac...,{'contexts': ['Programmed cell death (PCD) is ...,Results depicted mitochondrial dynamics in viv...,yes
1,16418930,Landolt C and snellen e acuity: differences in...,{'contexts': ['Assessment of visual acuity dep...,"Using the charts described, there was only a s...",no
2,9488747,"Syncope during bathing in infants, a pediatric...",{'contexts': ['Apparent life-threatening event...,"""Aquagenic maladies"" could be a pediatric form...",yes
3,17208539,Are the long-term results of the transanal pul...,{'contexts': ['The transanal endorectal pull-t...,Our long-term study showed significantly bette...,no
4,10808977,Can tailored interventions increase mammograph...,{'contexts': ['Telephone counseling and tailor...,The effects of the intervention were most pron...,yes


In [ ]:
pqa_unlabeled.head()

,pubid,question,context,long_answer
0,14499029,Is naturopathy as effective as conventional th...,{'contexts': ['Although the use of alternative...,Naturopathy appears to be an effective alterna...
1,14499049,Can randomised trials rely on existing electro...,"{'contexts': ['To estimate the feasibility, ut...",Routine data have the potential to support hea...
2,14499672,Is laparoscopic radical prostatectomy better t...,{'contexts': ['To compare morbidity in two gro...,The results of our non-randomized study show t...
3,14499773,Does bacterial gastroenteritis predispose peop...,{'contexts': ['Irritable bowel syndrome (IBS) ...,Symptoms consistent with IBS and functional di...
4,14499777,Is early colonoscopy after admission for acute...,{'contexts': ['Urgent colonoscopy has been pro...,No significant association is apparent between...


In [ ]:
# Step 1: Import Libraries
!pip install faiss-cpu
from sentence_transformers import SentenceTransformer
import faiss #FAISS library for similarity search and clustering
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration #transformers library to work with pretrained models like BERT
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.6 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter #for sentense embeddings

In [ ]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

documents = []

for idx, row in pqa_labeled.iterrows():
    # Extract context data (which is a dictionary)
    context_data = row['context']

    # Get the actual text contexts (stored in 'contexts' array)
    if 'contexts' in context_data:
        text_segments = context_data['contexts']

        # Process each text segment (some may be numpy strings)
        for segment in text_segments:
            # Convert numpy string to Python string if needed
            if isinstance(segment, np.str_):
                segment = str(segment)

            if isinstance(segment, str) and segment.strip():
                # Split into chunks
                chunks = text_splitter.split_text(segment)

                for chunk in chunks:
                    if chunk.strip():  # Only add non-empty chunks
                        documents.append({
                            'text': chunk,
                            'metadata': {
                                'pubmed_id': row['pubid'],
                                'question': row['question'],
                                'long_answer': row['long_answer'],
                                'final_decision': row['final_decision'],
                                'labels': context_data.get('labels', []),
                                'meshes': context_data.get('meshes', [])
                            }
                        })

print(f"Created {len(documents)} document chunks from PubMedQA dataset")
if documents:
    print("Sample chunk:", documents[0]['text'][:200] + "...")

Created 4509 document chunks from PubMedQA dataset
Sample chunk: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant co...


In [ ]:
# Initialize embedding model (using BAAI/bge which is state-of-the-art)
embedding_model = SentenceTransformer('BAAI/bge-base-en-v1.5')

# Generate embeddings for all chunks
texts = [doc['text'] for doc in documents]
embeddings = embedding_model.encode(texts, normalize_embeddings=True)

# Ensure embeddings is a 2D numpy array
if isinstance(embeddings, list):
    embeddings = np.array(embeddings)

# Check if embeddings is 1D (single embedding) and reshape if needed
if len(embeddings.shape) == 1:
    embeddings = embeddings.reshape(1, -1)  # Convert to 2D array with shape (1, embedding_dim)

# Create FAISS index for efficient similarity search
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print("FAISS index created with", index.ntotal, "document embeddings")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created with 4509 document embeddings


In [ ]:
!pip install rank_bm25 cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.5 MB/s eta 0:00:00


In [ ]:
from rank_bm25 import BM25Okapi
import cohere
import os

# Initialize BM25 with error handling
def initialize_bm25(documents):
    """
    Safely initialize BM25 with proper document validation

    Args:
        documents: List of document dictionaries with 'text' keys

    Returns:
        BM25Okapi instance or None if initialization fails
    """
    # Filter and validate documents
    valid_docs = []
    for doc in documents:
        if isinstance(doc.get('text'), str) and doc['text'].strip():
            tokens = [token for token in doc['text'].split() if token]  # Remove empty strings
            if tokens:  # Only add if we have tokens
                valid_docs.append(tokens)

    if not valid_docs:
        print("Warning: No valid documents for BM25 - all documents were empty after processing")
        return None

    try:
        bm25 = BM25Okapi(valid_docs)
        print(f"Initialized BM25 with {len(valid_docs)} documents")
        return bm25
    except Exception as e:
        print(f"BM25 initialization failed: {str(e)}")
        return None

# Initialize BM25 (with error handling)
bm25 = initialize_bm25(documents)

# Initialize Cohere for re-ranking
try:
    co = cohere.Client(os.environ.get("COHERE_API_KEY"))
    cohere_available = True
    print("Cohere reranker initialized successfully")
except:
    print("Cohere API key not found. Will use simple hybrid search without re-ranking.")
    cohere_available = False

def hybrid_search(query, top_k=5):
    """
    Hybrid search combining vector and keyword search
    """
    # Vector search
    query_embedding = embedding_model.encode(query, normalize_embeddings=True)
    D, I = index.search(np.array([query_embedding]), top_k*3)  # Get more initially

    # BM25 search (if available)
    if bm25 is not None:
        tokenized_query = query.split()
        bm25_scores = bm25.get_scores(tokenized_query)
        bm25_top_k_indices = np.argsort(bm25_scores)[-top_k*3:][::-1]
        combined_indices = set(I[0]).union(set(bm25_top_k_indices))
    else:
        # Fallback to just vector search if BM25 failed
        combined_indices = set(I[0])

    combined_docs = [documents[idx] for idx in combined_indices]

    # Re-rank with Cohere if available
    if cohere_available and len(combined_docs) > 1:
        try:
            rerank_docs = [doc['text'] for doc in combined_docs]
            rerank_results = co.rerank(
                query=query,
                documents=rerank_docs,
                top_n=top_k,
                model="rerank-english-v2.0"
            )
            final_results = []
            for result in rerank_results:
                doc_idx = list(combined_indices)[result.index]
                final_results.append(documents[doc_idx])
            return final_results[:top_k]
        except Exception as e:
            print(f"Cohere reranking failed: {e}. Falling back to simple hybrid.")

    # Fallback ranking by combining scores (if BM25 available)
    if bm25 is not None:
        tokenized_query = query.split()
        combined_docs.sort(
            key=lambda x: (
                0.5 * D[0][list(I[0]).index(documents.index(x))] if documents.index(x) in I[0] else 0
            ) + (
                0.5 * bm25.get_scores(tokenized_query)[documents.index(x)]
            ),
            reverse=True
        )
    else:
        # Pure vector search fallback
        combined_docs.sort(
            key=lambda x: D[0][list(I[0]).index(documents.index(x))] if documents.index(x) in I[0] else 0,
            reverse=True
        )

    return combined_docs[:top_k]

Initialized BM25 with 4509 documents
Cohere API key not found. Will use simple hybrid search without re-ranking.


In [ ]:
# Step 4: Generation with Step-Back Prompting
from transformers import RagTokenizer, RagSequenceForGeneration
import torch

# Initialize tokenizer and model (NO retriever)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import RagRetriever

class DummyRetriever(RagRetriever):
    def __init__(self):
        pass

    def retrieve(self, *args, **kwargs):
        return []


In [ ]:
def generate_answer(question, top_k=5):
    try:
        # 1. Retrieve documents
        retrieved_docs = hybrid_search(question, top_k=top_k)

        # 2. Prepare contexts
        contexts = [str(doc['text']) for doc in retrieved_docs
                    if doc and isinstance(doc.get('text'), (str, bytes))]

        if not contexts:
            return "No relevant medical context found."

        # 3. Combine question and context
        context_text = " ".join(contexts)
        input_text = f"Question: {question}\nContext: {context_text}"

        # 4. Tokenize
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )

        # Debug: check tokenizer output
        if inputs is None:
            print("Tokenizer returned None")
            return "Tokenizer failed to return inputs."

        if "input_ids" not in inputs or "attention_mask" not in inputs:
            print(f"Tokenizer output missing keys: {inputs.keys()}")
            return "Invalid tokenizer output."

        # 5. Generate
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4
        )

        if outputs is None or len(outputs) == 0:
            return "Model failed to generate an output."

        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        print(f"Error generating answer: {str(e)}")
        return "Failed to generate an answer due to internal error."


In [ ]:
questions = [
    "What is the mechanism of aspirin in heart attack prevention?",
    "Recommended metformin dosage for type 2 diabetes?",
    "Latest COVID-19 booster guidelines?",
    "Does aspirin helps with heartattacks?"
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: What is the mechanism of aspirin in heart attack prevention?
RAG Answer: the placement of the superficial cervical plexus block provides the same effective anesthesia
-----
Q: Recommended metformin dosage for type 2 diabetes?
RAG Answer: maximal feasible dosages of sulfonylurea and metformin (baseline glycosylated hemoglobin [HbA1c]=8.5%)
-----
Q: Latest COVID-19 booster guidelines?
RAG Answer: 2001 feeding recommendations for infants of HIV-infected mothers for a rural region in KwaZulu Natal, South Africa
-----
Q: Does aspirin helps with heartattacks?
RAG Answer: Aspirin has a positive effect on the survival of patients with SCD.
-----


In [ ]:
# Mechanism based / factual questions
questions = [
    "How does aspirin prevent heart attacks?",
    "What is the mechanism of action of metformin?",
    "How does insulin regulate blood sugar?",
    "What is the role of ACE inhibitors in hypertension?",
    "How do statins lower cholesterol?"
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: How does aspirin prevent heart attacks?
RAG Answer: Prophylactic fixation of the uninjured hip does not augment VCUG interpretation or contribute management-changing information but nonetheless carries a non-negligible effective dose.
-----
Q: What is the mechanism of action of metformin?
RAG Answer: anti-folate
-----
Q: How does insulin regulate blood sugar?
RAG Answer: sulfonylurea and metformin
-----
Q: What is the role of ACE inhibitors in hypertension?
RAG Answer: endothelin-1 may play a vital role in these hemodynamic changes
-----
Q: How do statins lower cholesterol?
RAG Answer: Preoperative statin therapy
-----


In [ ]:
# Dosage and Treatment Guidelines
questions = [
    "What is the typical dosage of metformin for adults with type 2 diabetes?",
    "How much ibuprofen is recommended for arthritis pain?",
    "Standard treatment protocol for H. pylori infection?",
    "Recommended antibiotic for bacterial pneumonia in children?"
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: What is the typical dosage of metformin for adults with type 2 diabetes?
RAG Answer: 30/70 twice daily or bedtime isophane (NPH) insulin plus continued sulfonylurea and metformin
-----
Q: How much ibuprofen is recommended for arthritis pain?
RAG Answer: doses of ergocalciferol are adequate for correction of VDDI and hyperparathyroidism
-----
Q: Standard treatment protocol for H. pylori infection?
RAG Answer: extensive surgery and wide spectrum antibiotics
-----
Q: Recommended antibiotic for bacterial pneumonia in children?
RAG Answer: Prophylactic inhaled sodium heparin
-----


In [ ]:
# Symptoms and disease understanding
questions = [
    "What are the warning signs of a heart attack in women?",
    "Common symptoms of hypothyroidism?",
    "What causes shortness of breath in heart failure patients?"
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: What are the warning signs of a heart attack in women?
RAG Answer: heart rate, QT interval, and relation between ventricular repolarization and heart rate
-----
Q: Common symptoms of hypothyroidism?
RAG Answer: depression, anxiety, and subjective complaints of memory problems
-----
Q: What causes shortness of breath in heart failure patients?
RAG Answer: extremes of hypotension in patients with ischaemic heart disease taking drugs to treat angina and heart failure
-----


In [ ]:
# Comparative clinical evidence
questions = [
    "Is metformin more effective than sulfonylureas?",
    "Does intermittent fasting improve insulin sensitivity?",
    "Which antidepressant has fewer side effects: SSRIs or SNRIs?"
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: Is metformin more effective than sulfonylureas?
RAG Answer: insulin monotherapy or insulin in addition to unchanged oral hypoglycemic medication in patients with type 2 diabetes in primary care
-----
Q: Does intermittent fasting improve insulin sensitivity?
RAG Answer: Fasting plasma glucose, 2-hour post oral glucose tolerance test plasma glucose, insulin sensitivity as well as body fat percentage with Framingham risk score are associated with cardiovascular risk.
-----
Q: Which antidepressant has fewer side effects: SSRIs or SNRIs?
RAG Answer: SSRIs
-----


In [ ]:
# Policy and Recommendation Based
questions = [
    "When should children receive the MMR vaccine?",
    "Does the CDC recommend RSV vaccination for adults over 60?",
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: When should children receive the MMR vaccine?
RAG Answer: 4-8 years
-----
Q: Does the CDC recommend RSV vaccination for adults over 60?
RAG Answer: no
-----


In [ ]:
# Less Suitable of Risky Queries
questions = [
    "What are the 2025 COVID-19 guidelines?", # Real time queries
    "How would stopping metformin affect diabetic neuropathy after 3 years?", # Long reasoning chains
    "What treatment should I take if I’m 65 with hypertension and asthma?", # Highly personalized medical questions
    "What does a CT scan showing ground-glass opacity indicate?" # Image of lab value interpretations
]

for q in questions:
    print(f"Q: {q}")
    print("RAG Answer:", generate_answer(q))
    print("-----")


Q: What are the 2025 COVID-19 guidelines?
RAG Answer: a) to what extent do HNC patients report fear of cancer recurrence? (b) to what extent are fears of recurrence manifestations of illness-related anxiety? (c) Are fears of recurrence closely related to current symptoms, disease, and treatment-related factors, or psychological/socio-demographic factors? (d) Are fears of recurrence closely related to current symptoms, disease, and treatment-related factors, or psychological/socio-demographic factors?
-----
Q: How would stopping metformin affect diabetic neuropathy after 3 years?
RAG Answer: no significant improvements were found in the metformin group when compared with the placebo group
-----
Q: What treatment should I take if I’m 65 with hypertension and asthma?
RAG Answer: infliximab
-----
Q: What does a CT scan showing ground-glass opacity indicate?
RAG Answer: benign and likely to be insignificant, benign and likely to be significant; and malignant
-----
